In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import time
from random import sample
import ipfshttpclient
from os import listdir
from os.path import isfile, join
import re
import praw

import substrateinterface
from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.exceptions import SubstrateRequestException

from substrate_helpers import set_substrate, set_delegate, make_call, addSchema, get_msa_id, get_signature, create_msa_with_delegator, mint_votes, mint_user, follow_user

/home/chia/miniconda3/lib/python3.8/site-packages/ipfshttpclient/client/__init__.py:73: VersionMismatch: Unsupported daemon version '0.13.0' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


In [3]:
reddit_creds = json.load(open(".reddit_creds.json", "r"))
reddit = praw.Reddit(
    client_id=reddit_creds["client_id"],
    client_secret=reddit_creds["client_secret"],
    password=reddit_creds["password"],
    user_agent=reddit_creds["user_agent"],
    username=reddit_creds["username"],
)

In [22]:
substrate = SubstrateInterface(
    url="ws://127.0.0.1:11946",
    ss58_format=42,
    type_registry_preset='kusama'
)
bob = Keypair.create_from_uri('//Bob')
set_delegate(bob)
set_substrate(substrate)
bob_msa_id = get_msa_id(bob)

In [8]:
r_all = reddit.subreddit('all')

In [9]:
client = ipfshttpclient.connect()

In [5]:
post_schema = "category STRING,username STRING,profile_pic STRING,title STRING,body STRING,url STRING,is_nsfw NUMERIC"
post_schemaId = addSchema(post_schema, wait_for_inclusion=True, wait_for_finalization=False)

comment_schema = "post_hash STRING,parent_hash STRING,parent_type STRING,username STRING,profile_pic STRING,body STRING"
comment_schemaId = addSchema(comment_schema, wait_for_inclusion=True, wait_for_finalization=False)

vote_schema = "post_hash STRING,parent_hash STRING,parent_type STRING,num_votes NUMERIC"
vote_schemaId = addSchema(vote_schema, wait_for_inclusion=True, wait_for_finalization=False)

user_schema = "msa_id NUMERIC,username STRING,password STRING,profile_pic STRING,wallet_ss58_address STRING"
user_schemaId = addSchema(user_schema, wait_for_inclusion=True, wait_for_finalization=False)

follow_schema = "protagonist_msa_id NUMERIC,antagonist_msa_id NUMERIC,event STRING"
follow_schemaId = addSchema(follow_schema, wait_for_inclusion=True, wait_for_finalization=False)

In [6]:
[post_schemaId, comment_schemaId, vote_schemaId, user_schemaId, follow_schemaId]

[155, 156, 159, 160, 161]

In [ ]:
path = "/tmp/"

In [20]:
def mint_comments(comments, post, post_data_hash):
    id_to_hash = {}
    for comment in comments:
        try:
            username = comment.author.name
            profile_pic = comment.author.icon_img
        except:
            username = "removed"
            profile_pic = "removed"
            
        password = 'password'
        user_wallet = Keypair.create_from_uri('//' + username + password)
        comment_user_msa_id, receipt_comment_user = mint_user(username, password, profile_pic, user_wallet)
            
        if comment.parent_id == post.name:
            parent_hash = post_data_hash
            parent_type = 'post'
        else:
            parent_hash = id_to_hash[comment.parent_id]
            parent_type = 'comment'
            
        comment_data = {
            "post_hash": post_data_hash,
            "parent_hash": parent_hash,
            "parent_type": parent_type,
            "username": username,
            "profile_pic": profile_pic,
            "body": comment.body,
        }
        
        comment_data_hash, receipt_comment = mint_data(comment_data, user_msa_id, path+'comments/' wait_for_inclusion=False, wait_for_finalization=False)
        
        receipt_comment = make_call("Messages", "add", call_params, bob, wait_for_inclusion=False, wait_for_finalization=False)
        print(comment_data_hash)
        
        ## comment votes ##
        receipt_ups = mint_votes(comment_user_msa_id, comment.ups, comment_data_hash, post_data_hash, 'comment')
        receipt_downs = mint_votes(comment_user_msa_id, comment.downs, comment_data_hash, post_data_hash, 'comment')
        print(comment.ups, comment.downs)
        
        ## FOLLOWS ##
        for follow_msa_id in sample(users, min(10, len(users))):
            follow_user(comment_user_msa_id, follow_msa_id)
            
        users.append(comment_user_msa_id)

In [23]:
schema_count = substrate.query(
    module='Schemas',
    storage_function='SchemaCount',
    params=[]
).value

while (True):
    for i, post in enumerate(r_all.top(time_filter='day')):
        ## USER ##
        try:
            username = post.author.name
            profile_pic = post.author.icon_img
        except:
            username = "removed"
            profile_pic = "removed"
            
        password = 'password'
        user_wallet = Keypair.create_from_uri('//' + username + password)
        user_msa_id, receipt_user = mint_user(username, password, profile_pic, user_wallet)

        ## POST ##
        post_data = {
            "category": post.subreddit.display_name,
            "username": username,
            "profile_pic": profile_pic,
            "title": post.title,
            "body": post.selftext,
            "url": post.url,
            "is_nsfw": post.over_18
        }

        post_data_hash, receipt_post = make_post(post_data, user_msa_id, path+'posts/', wait_for_inclusion=False, wait_for_finalization=False)
        print(post_data_hash)

        ## post votes ##
        receipt_ups = mint_votes(user_msa_id, post.ups, post_data_hash, post_data_hash, 'post')
        receipt_downs = mint_votes(user_msa_id, post.downs, post_data_hash, post_data_hash, 'post')
        print(post.ups, post.downs)

        ## COMMENT ##
        mint_comments(post.comments.list()[:10], post, post_data_hash)
        
        ## FOLLOWS ##
        for follow_msa_id in sample(users, min(10, len(users))):
            follow_user(user_msa_id, follow_msa_id)
            
        users.append(user_msa_id)
        
    print("waiting...")
    print()
    # sleep for a bit and then check again
    time.sleep(60*10)

Extrinsic '0xb5cafbb6c313f9261676610e390287c69a7b5739da89ae3b9b74386b113f3b62' sent and included in block 'None'
Extrinsic '0xd002df048664209bfd609f391cb13630d6e06006e77df33c439828fa94cc6db0' sent and included in block 'None'
Extrinsic '0x81bbb6952a69cc25cd539f35c975d31e026e3967e272587811bd9080eaab3ddc' sent and included in block 'None'
Extrinsic '0x68a1a0f067c6c36b3436a3db68774160c18392c5e29dcbfacdccde8fb72bbf20' sent and included in block 'None'
Extrinsic '0xb2f171be69c428488f4f4cad95adca69e2e3c5d822099675be711574e3e2d492' sent and included in block 'None'
Extrinsic '0xb3005c43b597882b171a7f5002a1bdf25a0ac11f111b28e14d3a419859cfd72c' sent and included in block 'None'
Extrinsic '0x65152b49fea7e38f6d34395618bffd635521d17f7bd5a46cdff1ad7c3111db1a' sent and included in block 'None'
Extrinsic '0x45f43a311919562d99b43feee8b6262edc6624fc97145e68168220b77b58a8f8' sent and included in block 'None'
Extrinsic '0x239052065565e66de58466c9b57cee27683e3c98fbf70220e41b0a4e1d94f1f8' sent and included

TypeError: 'NoneType' object is not subscriptable

In [39]:
b = reddit.redditor('abowlofspaghetti')

In [40]:
b.link_karma, b.comment_karma

(7003, 737, 0, 0)

In [47]:
b.submissions.new()

In [49]:
for t in b.submissions.new():
    break

In [52]:
t.selftext

"I'm not a huge fan of GBTC for Bitcoin but it's the only way to get exposure to crypto in my 401k. So I will be dumping all my GBTC whenever an ETF comes available. However, I feel like many people could do that and it'll tank the stock. Is that a possibility? I know it's supposed to represent btc value but it's constantly off by huge percentages."

In [32]:
[t for t in b.awardee_karma.top()]

[]

In [543]:
substrate.get_block()['header']['number']

9694

In [489]:
start_block = 5490

In [42]:
post_pattern = re.compile(f".*,.*,type,parent,data")

In [43]:
schema_count = substrate.query(
    module='Schemas',
    storage_function='SchemaCount',
    params=[]
).value

post_schemas = {}
for i in range(1, schema_count+1):
    schemaTemp = substrate.query(
        module='Schemas',
        storage_function='Schemas',
        params=[i]
    )
    if post_pattern.match(schemaTemp.value):
        print(schemaTemp.value)
        post_schemas[schemaTemp.value] = i

QmQzgLfEczTXa3rWFC4y7vJMECV7yFqbByhFPzUR9woUWs,interestingasfuck,type,parent,data


In [48]:
%%time
content_jsons = {}
for schema, schemaId in post_schemas.items():
    params = [
        schemaId,
        {
            "page_size": 10000,
            "from_block": 0,
            "to_block": 10000,
            "from_index": 0,
        }
    ]

    content = substrate.rpc_request(
        method='messages_getBySchema',
        params=params,
    )
    print(schema, content)
    if len(content['result']['content']) > 0:
        content_jsons[schema] = content['result']['content']

QmQzgLfEczTXa3rWFC4y7vJMECV7yFqbByhFPzUR9woUWs,interestingasfuck,type,parent,data {'jsonrpc': '2.0', 'result': {'content': [{'block_number': 34, 'data': '0x2c2c706f73742c2c516d517a674c6645637a5458613372574643347937764a4d454356377946716242796846507a555239776f555773', 'index': 0, 'msa_id': 1, 'signer': '5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty'}, {'block_number': 43, 'data': '0x2c2c706f73742c2c516d517a674c6645637a5458613372574643347937764a4d454356377946716242796846507a555239776f555773', 'index': 0, 'msa_id': 1, 'signer': '5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty'}], 'has_next': False}, 'id': 139}
CPU times: user 2.16 ms, sys: 0 ns, total: 2.16 ms
Wall time: 41.8 ms


In [45]:
content_jsons

{}

In [353]:
def add_comment(_data, _dict, _is_vote):
    for hsh in _dict.keys():
        if hsh == data[2]:
            if is_vote:
                if '-' in data[3]:
                    # downvote
                    _dict['data']['downvotes'] += data[3]
                else:
                    # upvote
                    _dict['data']['upvotes'] += data[3]
            else:
                _dict[data[3]] = {
                    'data': json.loads(client.cat(_data).decode()), 
                    'comments': {}
                }
                
                _dict['data'].update({"upvotes": 0, "downvotes": 0})
            return True
        result = add_comment(_data, _dict['comments'], _is_vote)
        if result:
            return True
    return False

In [354]:
posts = {}
for schema, contents in content_jsons.items():
    schema_items = schema.split(',')
    category = schema_items[0]
    post_hash = schema_items[1]
    posts[post_hash] = {'data': {"upvotes": 0, "downvotes": 0}, 'comments': {}}
    
    for content in contents:
        data = bytes.fromhex(content['data'][2:]).decode().split(',')
        # votes had numeric numbers instead of hashes
        is_vote = data[3].strip('-').isnumeric() 
        if is_vote:
            data[3] = int(data[3])
            
        if data[2] == '': 
            # empty means its post data
            posts[post_hash]['data'].update(json.loads(client.cat(data[3]).decode()))
        elif data[2] == post_hash: 
            # post_hash means its comment
            if is_vote:
                if data[3] < 0:
                    # downvote
                    posts[post_hash]['data']['downvotes'] += data[3]
                else:
                    # upvote
                    posts[post_hash]['data']['upvotes'] += data[3]
            else:
                posts[post_hash]['comments'][data[3]] = {
                    'data': json.loads(client.cat(data[3]).decode()), 
                    'comments': {}
                }
                posts[post_hash]['comments'][data[3]]['data'].update({"upvotes": 0, "downvotes": 0})
        else: 
            # neither means its comment of comment
            result = add_comment(data[3], posts[post_hash]['comments'], is_vote)
            if not result:
                print('couldnt find', data[2], json.loads(client.cat(data[3]).decode()))

In [355]:
from collections import OrderedDict
o = OrderedDict(sorted(posts.items(), key=lambda t: t[1]['data']['upvotes'], reverse=True))

In [357]:
posts_list = []
for k, v in OrderedDict(sorted(posts.items(), key=lambda t: t[1]['data']['upvotes'], reverse=True)).items():
    posts_list.append({k:v})